### 结果对比

In [3]:
from runner.runner import train_test
import torch
from runner.runner import visualize_result

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

algos = [
        # "SIDLinear",
        "STID",
        # "PDFormer"
        ]

datas = [
    "PEMS-BAY",
    "PEMS08",
    "PEMS04",
    "METR-LA",

    # "T-Drive",
    # "CHIBike",
    # "NYCTaxi"
    ]
epochs = 1
val_interval = 1
out_dir = "./checkpoints"

train_test(datas,algos,epochs,val_interval,out_dir,device)
# visualize_result(algos,epochs,out_dir,device,data=datas[0])

2024-07-21 13:19:15,179 - PEMS08_STID_1 - INFO - run PEMS08_STID
2024-07-21 13:19:16,205 - PEMS08_STID_1 - INFO - parameter count of STID:33388
2024-07-21 13:19:16,207 - PEMS08_STID_1 - INFO - model config: 
NAME: STID_PEMS08
ARCH: <class 'baselines.STID.stid_arch.STID'>
LOSS: masked_mae
PARAM:
  num_nodes: 170
  input_len: 12
  input_dim: 1
  embed_dim: 16
  output_len: 12
  num_layer: 3
  if_node: True
  node_dim: 16
  if_T_i_D: True
  if_D_i_W: True
  temp_dim_tid: 16
  temp_dim_diw: 16
  time_of_day_size: 288
  day_of_week_size: 7
  type: FC
  concat: True

2024-07-21 13:19:16,207 - PEMS08_STID_1 - INFO - dataset config:  
NAME: PEMS08
NUM_NODES: 170
STEPS_PER_DAY: 288
READ_DATA_FUNC: read_npz
TOD: True
DOW: True
DOM: False
DOY: False
OUTPUT_DIR: datasets/PEMS08
DATA_FILE_PATH: datasets/raw_data/PEMS08/PEMS08.npz
GRAPH_FILE_PATH: datasets/raw_data/PEMS08/adj_PEMS08.pkl
DTW_FILE_PATH: datasets/raw_data/PEMS08/PEMS08_dtw.npy
SHMX_FILE_PATH: datasets/raw_data/PEMS08/PEMS08_sh_mx.npy
P

{'PEMS08_STID': <utils.meter_pool.MeterPool at 0x7f928c62ca30>}

### copy result

In [2]:
import os  
import shutil
result_dir = "result2"
if not os.path.exists(result_dir):
    os.makedirs(result_dir)

for root, dirs, files in os.walk("checkpoints2"):
    for file in files:
        if ".log" in file:
            ds, algo, epoch = root.split("/")[1].split("_")
            file_path = os.path.join(root, file)
            shutil.copy(file_path, os.path.join(result_dir,f"{ds}_{algo}_{epoch}.log")) 


### plot train loss

In [28]:
import pandas as pd
import os  
import plotly
import plotly.io as pio
import plotly.graph_objects as go
pio.templates.default = 'plotly_white'

def get_train_loss(filename):
    loss = []
    with open(filename,"r") as f:
        for line in f:
            if "train_MAE" in line:
                loss.append(float(line.split("train_MAE: ")[1].split(',')[0]))
    return loss

def get_test_mae(filename):
    loss = ""
    with open(filename,"r") as f:
        for line in f:
            if "test_MAE" in line:
                loss = float(line.split("test_MAE: ")[1].split(',')[0])
    return loss

def plot_train_loss(result_dir):
    results = {}
    for root, dirs, files in os.walk(result_dir):
        for file in files:
            f = file.split("_")
            if len(f) == 4:
                ds = "_".join([f[0],f[2]])
            else:
                ds = f[0]
            file_path = os.path.join(root, file)
            if ds in results.keys():
                results[ds].append(file_path)
            else:
                results[ds] = [file_path]

    mae_all = {}
    method_List = []
    for ds, result in results.items():
        mae_all[ds] = []
        for ret in result:
            mae_all[ds].append(get_test_mae(ret))
            method = os.path.basename(ret).split("_")[1]
            if method not in method_List:
                method_List.append(method)

    df = pd.DataFrame(mae_all, index=method_List)

    for ds, result in results.items():
        fig = go.Figure()
        for i, ret in enumerate(result):
            width = 2 if ret.find("STDCN")>0 else 2 if ret.find("PDFormer")>0 else 1
            color = 'red' if ret.find("STDCN")>0 else 'black' if ret.find("PDFormer")>0 else plotly.colors.sequential.Blues_r[i%9]
            fig.add_trace(go.Scatter(y=get_train_loss(ret), mode='lines', 
                                    name="_".join(os.path.basename(ret).split("_")[:2]), 
                                     marker={"color":color},
                                    line={"width":width}))

        fig.update_layout(
                        title = ds,
                        height=400,
                        width=600,
                        yaxis=dict(title='loss', titlefont=dict(size=10)
                                    , tickfont=dict(size=10),
                                    showgrid=False,
                                    ),
                        xaxis=dict(title='epochs', titlefont=dict(size=10)
                                    , tickfont=dict(size=10), 
                                    showgrid=False,
                                    ) ,
                        xaxis_tickangle=-45)

        fig.show()
    
    return df


In [29]:
df = plot_train_loss("result2")
df

,METR-LA,PEMS-BAY,PEMS04,PEMS08
AGCRN,4.72,1.669,21.003,15.92
BGSLF,5.691,2.09,23.293,19.465
D2STGNN,4.411,1.57,17.937,13.752
DCRNN,,,,
DGCRN,4.437,1.595,18.674,14.351
GraphWaveNet,4.55,1.657,19.346,14.945
PDFormer,4.549,1.608,18.345,13.539
STDCN,4.616,1.672,18.098,13.951
StemGNN,4.791,1.808,19.961,15.357
STGCN,4.608,1.616,19.592,16.14


In [30]:
df = plot_train_loss("result")
df

,CHIBike_inflow,CHIBike_outflow,METR-LA,NYCTaxi_inflow,NYCTaxi_outflow,PEMS-BAY,PEMS04,PEMS08,T-Drive_inflow,T-Drive_outflow
DCRNN,,,,,,,,,,
PDFormer,0.339,0.336,4.609,10.04,9.322,1.644,18.438,14.109,14.529,14.671
SIDLinear,0.345,0.367,4.528,10.494,9.572,1.736,18.556,14.492,13.729,13.838
STDCN,0.347,0.341,4.656,9.423,9.451,1.67,18.245,13.528,13.452,13.435
StemGNN,0.345,0.345,4.795,10.357,10.357,1.717,20.765,15.263,19.969,19.969
STID,0.34,0.334,4.704,10.367,9.518,1.729,18.61,14.415,13.726,13.79


### 数据集展示

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [40]:
# PEMS-BAY
df = pd.read_hdf("datasets/raw_data/PEMS-BAY/PEMS-BAY.h5")
data = np.expand_dims(df.values, axis=-1)

fig = go.Figure()
for i in [1,5,10,20,100]:
    fig.add_trace(go.Scatter(y=data[:10000,i].reshape(-1), name=f"series{i}"))
fig.update_layout(
                height=400,
                width=800,)
fig.show()

In [43]:
df = pd.read_csv("datasets/raw_data/NYCTaxi/NYCTaxi.grid")
data = []
for i in range(0, df.shape[0], df.shape[0]//75):
    data.append(df[i:i+df.shape[0]//75]['inflow'].values)
data = np.array(data)

fig = go.Figure()
for i in [10,20,30,40,50]:
    fig.add_trace(go.Scatter(y=data[i,:1000], name=f"series{i}"))
fig.update_layout(
                height=400,
                width=800,)
fig.show()

In [45]:
data = np.load("datasets/raw_data/PEMS08/PEMS08.npz")["data"][...,0]

fig = go.Figure()
for i in [10,20,30,40,50]:
    fig.add_trace(go.Scatter(y=data[:5000,i].reshape(-1), name=f"series{i}"))
fig.update_layout(
                height=400,
                width=800,)
fig.show()

***

In [4]:
from config.cfg_algorithm import CFG_MODEL
from config.cfg_general import CFG_GENERAL
from config.cfg_dataset import CFG_DATASET
import os
from runner.builder import build_model
from data.data_loader import build_dataset
from utils.__init__ import config_md5
import torch

data = "PEMS08"
algo = "SIDLinear"

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
epochs = 10
val_interval = 1
out_dir = "./checkpoints"


model_args = CFG_MODEL[algo](data, CFG_DATASET[data]["NUM_NODES"],
                        CFG_GENERAL.DATASET.HISTORY_SEQ_LEN,
                        CFG_GENERAL.DATASET.FUTURE_SEQ_LEN,
                        not CFG_GENERAL.DATASET.NORM_EACH_CHANNEL,
                        CFG_DATASET[data].STEPS_PER_DAY)

md5 = config_md5(CFG_GENERAL, CFG_DATASET[data], model_args)
ckpt_save_dir = os.path.join(out_dir, "_".join([data, algo, str(epochs)]),md5)
if not os.path.isdir(ckpt_save_dir):
    os.makedirs(ckpt_save_dir)

models = build_model(CFG_GENERAL, model_args)
datasets = build_dataset(CFG_GENERAL, CFG_DATASET[data])
future_data, history_data = next(iter(datasets["train"]))
future_data, history_data = future_data.to(device), history_data.to(device)

model = models["model"].to(device)
out = model(history_data,history_data,0,0,False)
out.shape,future_data.shape, history_data.shape

(torch.Size([32, 12, 170, 1]),
 torch.Size([32, 12, 170, 3]),
 torch.Size([32, 12, 170, 3]))

### SIDLinear1: 原始

In [9]:
import torch
from torch import nn
import math
class moving_avg(nn.Module):
    """
    Moving average block to highlight the trend of time series
    """
    def __init__(self, kernel_size, stride):
        super(moving_avg, self).__init__()
        self.kernel_size = kernel_size
        self.avg = nn.AvgPool1d(kernel_size=kernel_size, stride=stride, padding=0)

    def forward(self, x):
        # padding on the both ends of time series
        front = x[:, 0:1, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        end = x[:, -1:, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        x = torch.cat([front, x, end], dim=1)
        x = self.avg(x.permute(0, 2, 1))
        x = x.permute(0, 2, 1)
        return x


class series_decomp(nn.Module):
    """
    Series decomposition block
    """
    def __init__(self, kernel_size):
        super(series_decomp, self).__init__()
        self.moving_avg = moving_avg(kernel_size, stride=1)

    def forward(self, x):
        moving_mean = self.moving_avg(x)
        res = x - moving_mean
        return res, moving_mean

class series_decomp2(nn.Module):
    """
    Series decomposition block
    """
    def __init__(self, kernel_size):
        super(series_decomp2, self).__init__()
        self.moving_avg = moving_avg(kernel_size, stride=1)

    def forward(self, x):
        B,D,N,L = x.shape
        x = x.reshape(B*D,N,L).transpose(1,2)
        moving_mean = self.moving_avg(x)
        res = x - moving_mean
        return res.transpose(1,2).reshape(B,D,N,L), moving_mean.transpose(1,2).reshape(B,D,N,L)

class PositionalEncoding(nn.Module):
    def __init__(self, embed_dim, max_len=100):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, embed_dim).float()
        pe.require_grad = False

        position = torch.arange(0, max_len).float().unsqueeze(1)
        div_term = (torch.arange(0, embed_dim, 2).float() * -(math.log(10000.0) / embed_dim)).exp()

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return self.pe[:, :x.size(1)].unsqueeze(-1).expand_as(x).detach()
    
class DataEmbedding(nn.Module):
    def __init__(self, c_in, d_model, num_nodes=None, time_of_day_size=None, day_of_week_size=None, dropout=0.1):
        super(DataEmbedding, self).__init__()
        self.time_of_day_size = time_of_day_size
        self.day_of_week_size = day_of_week_size
        self.num_nodes = num_nodes

        self.value_emb = nn.Conv2d(in_channels=c_in, out_channels=d_model, kernel_size=(1,1), bias=True)
        self.position_encoding = PositionalEncoding(self.num_nodes)
        if self.num_nodes is not None:
            self.node_emb = nn.Parameter(torch.empty(self.num_nodes, d_model))
            nn.init.xavier_uniform_(self.node_emb)
        if self.time_of_day_size is not None:
            self.time_in_day_emb = nn.Parameter(torch.empty(self.time_of_day_size, self.num_nodes))
            nn.init.xavier_uniform_(self.time_in_day_emb)
        if self.day_of_week_size is not None:
            self.day_in_week_emb = nn.Parameter( torch.empty(self.day_of_week_size, self.num_nodes))
            nn.init.xavier_uniform_(self.day_in_week_emb)
        
    def forward(self, history):

        bs = history.shape[0]
        value_embed = self.value_emb(history[...,[0]])
        pos_emb = self.position_encoding(history[...,[0]])

        if self.time_of_day_size is not None:
            day_embed = self.time_in_day_emb[(history[:,:,0,1]*self.time_of_day_size).long()]
        else:
            day_embed = 0
        if self.day_of_week_size is not None:
            week_embed = self.day_in_week_emb[(history[:,:,0,2]*self.day_of_week_size).long()]
        else:
            week_embed = 0
        if self.num_nodes is not None:
            node_embed = self.node_emb.unsqueeze(0).unsqueeze(-1).expand(bs,-1,-1,-1).transpose(1,2)
        else:
            node_embed = 0
        x = torch.cat([value_embed] + [pos_emb] + [day_embed.unsqueeze(-1)] + [week_embed.unsqueeze(-1)] + [node_embed],dim=1)


        return x
    

class MultiLayerPerceptron(nn.Module):
    def __init__(self, input_dim, hidden_dim) -> None:
        super().__init__()
        self.fc1 = nn.Conv2d(
            in_channels=input_dim,  out_channels=hidden_dim, kernel_size=(1, 1), bias=True)
        self.fc2 = nn.Conv2d(
            in_channels=hidden_dim, out_channels=hidden_dim, kernel_size=(1, 1), bias=True)
        self.act = nn.ReLU()
        self.drop = nn.Dropout(p=0.15)

    def forward(self, input_data: torch.Tensor) -> torch.Tensor:
        hidden = self.fc2(self.drop(self.act(self.fc1(input_data))))      # MLP
        hidden = hidden + input_data                           # residual
        return hidden


class SIDLinear(nn.Module):


    def __init__(self, **model_args):
        super().__init__()
        # attributes
        self.num_nodes = model_args["num_nodes"]
        self.node_dim = model_args["node_dim"]
        self.input_len = model_args["input_len"]
        self.input_dim = model_args["input_dim"]
        self.embed_dim = model_args["embed_dim"]
        self.output_len = model_args["output_len"]
        self.num_layer = model_args["num_layer"]
        self.temp_dim_tid = model_args["temp_dim_tid"]
        self.temp_dim_diw = model_args["temp_dim_diw"]
        self.time_of_day_size = model_args["time_of_day_size"]
        self.day_of_week_size = model_args["day_of_week_size"]

        self.if_time_in_day = model_args["if_T_i_D"]
        self.if_day_in_week = model_args["if_D_i_W"]
        self.if_spatial = model_args["if_node"]

        self.decom = series_decomp2(5)

        self.hidden_dim = self.embed_dim+self.embed_dim * \
            int(self.if_spatial)+self.input_len*int(self.if_day_in_week) + \
            self.input_len*int(self.if_time_in_day) +self.input_len
        
        self.blocks = 3
        self.emb = DataEmbedding(self.input_len,self.embed_dim,self.num_nodes,self.time_of_day_size,self.day_of_week_size)        
        # self.block_season = nn.ModuleList()
        self.block_trend = nn.ModuleList()
        for i in range(self.blocks):
            # self.block_season.append(MultiLayerPerceptron(self.hidden_dim, self.hidden_dim))
            self.block_trend.append(MultiLayerPerceptron(self.hidden_dim, self.hidden_dim))
        
        # regression
        self.regression_layer = nn.Conv2d(
            in_channels=self.hidden_dim, out_channels=self.output_len, kernel_size=(1, 1), bias=True)
        
        # for m in self.modules():
        #     if isinstance(m, nn.Conv2d):
        #         nn.init.xavier_normal(m.weight)

    def forward(self, history_data: torch.Tensor, future_data: torch.Tensor, batch_seen: int, epoch: int, train: bool, **kwargs) -> torch.Tensor:
        x = self.emb(history_data)
        # residual,trend = self.decom(x)
        residual,trend = 0,x
        for i in range(self.blocks):
            hidden = self.block_trend[i](trend)
            # residual = self.block_season[i](residual)
            senson,trend = self.decom(hidden)
            residual += senson
        prediction = self.regression_layer(trend)
        return prediction
    
args = {
        "num_nodes":170,
        "input_len": 12,
        "input_dim": 1,
        "embed_dim": 32,
        "output_len": 12,
        "num_layer": 3,
        "if_node": True,
        "node_dim": 32,
        "if_T_i_D": True,
        "if_D_i_W": True,
        "temp_dim_tid": 32,
        "temp_dim_diw": 32,
        "time_of_day_size": 288,
        "day_of_week_size": 7,
        "type":"FC",                # [FC,CONV1D,CONV2D]
        "concat":True
}
model = SIDLinear(**args)
model = model.to(device)
model(history_data,history_data,0,0,False).shape

torch.Size([32, 12, 170, 1])

### SIDLinear2：
- 嵌入拼接 -> 性能更差了
- 嵌入叠加 -> 接近PDFormer

In [61]:
import torch
from torch import nn
    
class DataEmbedding(nn.Module):
    def __init__(self, d_model, num_nodes=None, time_of_day_size=None, day_of_week_size=None, dropout=0.1):
        super(DataEmbedding, self).__init__()
        self.time_of_day_size = time_of_day_size
        self.day_of_week_size = day_of_week_size
        self.num_nodes = num_nodes

        self.value_emb = nn.Conv2d(in_channels=1, out_channels=d_model, kernel_size=(1,1), bias=True)
        if self.num_nodes is not None:
            self.node_emb = nn.Parameter(torch.empty(self.num_nodes, d_model))
            nn.init.xavier_uniform_(self.node_emb)
        if self.time_of_day_size is not None:
            self.time_in_day_emb = nn.Parameter(torch.empty(self.time_of_day_size, d_model))
            nn.init.xavier_uniform_(self.time_in_day_emb)
        if self.day_of_week_size is not None:
            self.day_in_week_emb = nn.Parameter( torch.empty(self.day_of_week_size, d_model))
            nn.init.xavier_uniform_(self.day_in_week_emb)
        
    def forward(self, history):

        bs,in_len,_,_ = history.shape
        x = self.value_emb(history[...,[0]].permute(0,3,1,2)).permute(0,2,3,1)

        if self.time_of_day_size is not None:
            x += self.time_in_day_emb[(history[:,:,0,1]*self.time_of_day_size).long()].unsqueeze(2).expand(-1,-1,self.num_nodes,-1)

        if self.day_of_week_size is not None:
            x += self.day_in_week_emb[(history[:,:,0,2]*self.day_of_week_size).long()].unsqueeze(2).expand(-1,-1,self.num_nodes,-1)

        if self.num_nodes is not None:
            x += self.node_emb.unsqueeze(0).unsqueeze(1).expand(bs,in_len,-1,-1)

        return x.permute(0,3,1,2)

class MultiLayerPerceptron(nn.Module):
    def __init__(self, input_dim, hidden_dim) -> None:
        super().__init__()
        self.fc1 = nn.Conv2d(
            in_channels=input_dim,  out_channels=hidden_dim, kernel_size=(3, 1), bias=True, padding='same')
        self.fc2 = nn.Conv2d(
            in_channels=hidden_dim, out_channels=hidden_dim, kernel_size=(3, 1), bias=True, padding='same')
        self.act = nn.ReLU()
        self.drop = nn.Dropout(p=0.15)

    def forward(self, input_data: torch.Tensor) -> torch.Tensor:
        hidden = self.fc2(self.drop(self.act(self.fc1(input_data))))      # MLP
        hidden = hidden + input_data                           # residual
        return hidden

class SIDLinear(nn.Module):


    def __init__(self, **model_args):
        super().__init__()
        # attributes

        self.d_model = model_args["d_model"]
        self.num_nodes = model_args["num_nodes"]
        self.input_len = model_args["input_len"]
        self.output_len = model_args["output_len"]
        self.num_layer = model_args["num_layer"]
        self.time_of_day_size = model_args["time_of_day_size"]
        self.day_of_week_size = model_args["day_of_week_size"]

        self.if_time_in_day = model_args["if_T_i_D"]
        self.if_day_in_week = model_args["if_D_i_W"]
        self.if_spatial = model_args["if_node"]
        
        self.emb = DataEmbedding(self.d_model,self.num_nodes,self.time_of_day_size,self.day_of_week_size)        
        self.block = nn.ModuleList()
        for i in range(self.num_layer):
            self.block.append(MultiLayerPerceptron(self.d_model, self.d_model))
        self.end_conv1 = nn.Conv2d(in_channels=self.d_model, out_channels=1, kernel_size=(1,1), bias=True)
        self.end_conv2 = nn.Conv2d(in_channels=self.input_len, out_channels=self.output_len, kernel_size=(1,1), bias=True)
        


    def forward(self, history_data: torch.Tensor, future_data: torch.Tensor, batch_seen: int, epoch: int, train: bool, **kwargs) -> torch.Tensor:
        x = self.emb(history_data)
        for i in range(self.num_layer):
            x = self.block[i](x)
        
        out1 = self.end_conv1(x)
        prediction = self.end_conv2(out1.permute(0,2,3,1))
        return prediction
    
args = {
        "num_nodes":170,
        "d_model": 64,
        "input_len": 12,
        "output_len": 12,
        "num_layer": 3,
        "if_node": True,
        "if_T_i_D": True,
        "if_D_i_W": True,
        "time_of_day_size": 288,
        "day_of_week_size": 7,
}
model = SIDLinear(**args)
model = model.to(device)
model(history_data,history_data,0,0,False).shape

torch.Size([32, 12, 170, 1])